<a href="https://colab.research.google.com/github/tanish36/Tensorflow-Projects/blob/master/UD_Sentiment_Analysis_using_ELMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Making a Twitter Sentiment Analysis using The Elmo text Embemding from tfhub

IMPORTING THE REQUIRED LIBRARIES <br>

Using Tensorflow 1.15 as the Elmo Model used here is not supported in the Tensorflow 2.x but will be in the the future

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

<h2>Importing the required file

Importing the file of the twitter tweets from 2016 Presidential debate using the Pandas Library for it

In [0]:
import pandas as pd
df =  pd.read_csv("https://raw.githubusercontent.com/tanish36/Tensorflow-Projects/master/Sentiment.csv",encoding="latin")

Using the df.info() to check the info of the iported file that we are going to use <br>we can see that their are 13871 total entries in the id block and the different data type of the of the each column 

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13871 entries, 0 to 13870
Data columns (total 21 columns):
id                           13871 non-null int64
candidate                    13775 non-null object
candidate_confidence         13871 non-null float64
relevant_yn                  13871 non-null object
relevant_yn_confidence       13871 non-null float64
sentiment                    13871 non-null object
sentiment_confidence         13871 non-null float64
subject_matter               13545 non-null object
subject_matter_confidence    13871 non-null float64
candidate_gold               28 non-null object
name                         13871 non-null object
relevant_yn_gold             32 non-null object
retweet_count                13871 non-null int64
sentiment_gold               15 non-null object
subject_matter_gold          18 non-null object
text                         13871 non-null object
tweet_coord                  21 non-null object
tweet_created                13871 no

Getting the info on our dataset and labelling the sentiments as <br>
1. Neutral as 0<br>
2. Positive as 1<br>
3. Negative as 2 <br>

we are doing the encoding as we are using the logistic Regression model for the implementation which predicts value on the basis of the numbers

In [0]:
df.loc[df["sentiment"]=='Neutral',"sentiment"]=0
df.loc[df["sentiment"]=='Negative',"sentiment"]=1
df.loc[df["sentiment"]=='Positive',"sentiment"]=2
df.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,name,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,0,0.6578,None of the above,1.0000,NaN,I_Am_Kenzi,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,2,0.6333,None of the above,1.0000,NaN,PeacefulQuest,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,0,0.6629,None of the above,0.6629,NaN,PussssyCroook,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,2,1.0000,None of the above,0.7039,NaN,MattFromTexas31,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,2,0.7045,None of the above,1.0000,NaN,sharonDay5,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


<h2>Making a list of data for training

making a training dat which we will feed into the tf hub ELMO model for generating the feature vectors we are usinf only 131 tweet fromt the datset 

In [0]:
X = np.array(df["text"])
y = np.array(df["sentiment"])
l=[]
for i in range (131):
  l.append(df['text'][i])

<h2>Test Dataset

Using the two set of tweet for testing the data after making the model

In [0]:
new_text =  ['RT @FrankLuntz: Before the #GOPDebate, 14 focus groupers said they did not have favorable view of Trump.',
             'Chris Wallace(D) to be the 2nd worst partisan pontificating asshole "moderating" #GOPDebate @megynkelly'
            ]

<h2>Spliting the data into the training and testing set

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['sentiment'], test_size=0.2)

Using the Tensorflow hub library for elmo

importing the Elmo Model from TF HUB 

In [0]:
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

using the Elmo Model for Generarting the feature set for the taken dataset

The output is a 3 dimensional tensor of shape (131, 28, 1024):

The first dimension of this tensor represents the number of training samples. This is 131 in our case<br>
The second dimension represents the maximum length of the longest string in the input list of strings.<br>
The third dimension is equal to the length of the ELMo vector which is 1024

In [0]:
embeddings = elmo(l, signature="default", as_dict=True)["elmo"]

embeddings.shape

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


TensorShape([Dimension(131), Dimension(28), Dimension(1024)])

Making the elmo_vector function which will process the data to generate the elmo model feature set
and return the mean of the feature set result

In [0]:
def elmo_vectors(x):
  embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

Doing the preprocessing on the data for Predictions

In [0]:
elmo_train = [elmo_vectors(l) ]


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Concatenate the result of the elmo train in the single array int he next step

In [0]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)

defing the next variable for the logistic regression as yy 

In [0]:
yy=df['sentiment'].iloc[:131]
yy

0      0
1      2
2      0
3      2
4      2
      ..
126    1
127    2
128    0
129    1
130    1
Name: sentiment, Length: 131, dtype: int64

<h2>Using the Logistic Regression for Analysis and training



In [0]:
print(yy.shape,elmo_train_new.shape)

(131,) (131, 1024)


In [0]:
regressor = tf.estimator.LogisticRegressionHead()

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

lreg = LogisticRegression()
lreg.fit(elmo_train_new, yy)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
j=[df['text'][i] for i in range(132,184)]
el = [elmo_vectors(j)]
elmotestnew = np.concatenate(el, axis = 0)
#preds_valid = lreg.predict(xtest)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


<h1>Predicting the value


In [0]:
preds_valid = lreg.predict(elmotestnew)

In [0]:
lab=[df['sentiment'][i] for i in range(132,184)]
lab

[2,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 2,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 2,
 0,
 1,
 2,
 1,
 1]

Compairing the predicted value and real value using f1 score

In [0]:
from sklearn.metrics import f1_score
#preds_valid
f1_score(lab, preds_valid,average='micro')

0.6538461538461539

Predicting the value for a small dataset of two length

In [0]:
el1 = [elmo_vectors(new_text)]
elmotestnew1 = np.concatenate(el1, axis = 0)
jj=lreg.predict(elmotestnew1)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


<h2>Predicted Result

In [0]:
for i in jj:
  if i==0:
    print("NEUTRAL")
  elif i==1:
    print("Positive")
  elif i==2:
    print("Negative")    

Negative
Positive


In [0]:
for i in jj:
  print(i)

2
1


REFERENCES<BR>
1.https://towardsdatascience.com/transfer-learning-using-elmo-embedding-c4a7e415103c <br>
2.https://tfhub.dev/google/elmo/3 <br>
3.https://medium.com/learning-machine-learning/introduction-to-tensorflow-estimators-part-1-39f9eb666bc7